# Imports

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import re

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
from matplotlib.patches import Polygon
from matplotlib.cbook import boxplot_stats
from sklearn.preprocessing import MinMaxScaler

# Definitions

In [ ]:
def bp_stats(df, columns, names):
  # Define lists for storing the boxplot statistics
  q0s = []
  q1s = []
  meds = []
  q3s = []
  q4s = []

  # Loop through each column and create a boxplot
  for i, col in enumerate(columns):
      # Get the boxplot statistics for the current column
      data = df[col]
      stats = boxplot_stats(data)

      # Append the statistics to their respective lists
      q0s.append(stats[0]['whislo'])
      q1s.append(stats[0]['q1'])
      meds.append(stats[0]['med'])
      q3s.append(stats[0]['q3'])
      q4s.append(stats[0]['whishi'])

  # Create a new dataframe with the boxplot statistics
  df_bp_stats = pd.DataFrame({
      'category': names,
      'lower_whisker': q0s,
      'lower_quartile': q1s,
      'median': meds,
      'upper_quartile': q3s,
      'upper_whisker': q4s
  })

  df_bp_stats.iloc[:, 1:] = df_bp_stats.iloc[:, 1:].applymap(lambda x: f"{x:.0f}" if isinstance(x, (int, float)) and x > 100 else (f"{x:.1f}" if isinstance(x, (int, float)) and x > 9 else f"{x:.2f}"))


  return df_bp_stats

In [ ]:
def get_box_plot_data(data, name):
  # Get box plot statistics
  stats = boxplot_stats(data)

  # Extract relevant statistics
  q0 = stats[0]['whislo']
  q1 = stats[0]['q1']
  median = stats[0]['med']
  q2 = stats[0]['q3']
  q4 = stats[0]['whishi']

  IQR = q3-q1
  xmin, xmax = np.percentile(data, [0.01, 99.99])

  # Create a DataFrame with the statistics
  df_stats = pd.DataFrame({#'view': name,
                          'lower_whisker': q0,
                          'lower_quartile': q1,
                          'median': median,
                          'upper_quartile': q2,
                          'upper_whisker': q4}, index=[0])

  # Print the DataFrame
  print(df_stats)
  return q0, q1, median, q3, q4, IQR, xmin, xmax

### Def performance levels and labels finding

#### Daylight

In [ ]:
def find_daylight_level(daylight_median, room_type):
    if daylight_median >= 750:
        daylight_level = 'high'
    
    elif daylight_median >= 500:
        daylight_level = 'med'
    
    elif daylight_median >= 300:
        daylight_level = 'low'
    
    elif daylight_median < 300:
        if room_type in ('ROOM', 'BEDROOM'):
            if daylight_median >= 100:
                daylight_level = 'min'
            else:
                daylight_level = 'insufficient'
        elif room_type in ('LIVING_ROOM', 'DINING'):
            if daylight_median >= 150:
                daylight_level = 'min'
            else:
                daylight_level = 'insufficient'
        elif room_type == 'KITCHEN':
            if daylight_median >= 100:
                daylight_level = 'min'
            else:
                daylight_level = 'insufficient'
    
    return daylight_level

In [ ]:
def find_daylight_label(group):
    daylight_level_median = group['daylight_level_num'].median()
    daylight_level_min = group['daylight_level_num'].min()

    if daylight_level_min == 0:
        daylight_label = 'F'
    elif daylight_level_min == 4:
        daylight_label = 'A'  
    elif daylight_level_min == 1:
        if daylight_level_median >= 2:
            daylight_label = 'D'
        else: 
            daylight_label = 'E'
    elif daylight_level_min == 2:
        if daylight_level_median >= 3:
            daylight_label = 'C'
        elif daylight_level_median >= 2:
            daylight_label = 'D'
        else: 
            daylight_label = 'E'
    elif daylight_level_min == 3:
        if daylight_level_median >= 4:
            daylight_label = 'B'
        elif daylight_level_median >= 3:
            daylight_label = 'C'
        elif daylight_level_median >= 2:
            daylight_label = 'D'
        else: 
            daylight_label = 'E'
    else:
        daylight_label = None
    #print(f'min={daylight_level_min} median={daylight_level_median} label={daylight_label}')
    return daylight_label

In [ ]:
# Define the mapping dictionary
mapping_daylight_level = {
    'insufficient': 0,
    'min': 1,
    'low': 2,
    'med': 3,
    'high': 4
}

# Define the mapping dictionary
mapping_daylight_label = {
    'A': 1,
    'B': 0.8,
    'C': 0.6,
    'D': 0.4,
    'E': 0.2,
    'F': 0
}

#### View

In [ ]:
def count_visible_views(row):
    columns_to_check = ['view_ground_p80', 'view_landscape_p80', 'view_sky_p80']
    count = sum(1 for col in columns_to_check if row[col] >= 0.477)
    return count

def find_view_level(landscape_visible, nr_layers):
    if landscape_visible == 'no':
        view_level = 'insufficient'
    elif nr_layers == 1:
        view_level = 'min'
    elif nr_layers == 2:
        view_level = 'med'
    elif nr_layers == 3:
        view_level = 'high'

    return view_level

In [ ]:
def find_view_label(group):
    view_level_median = group['view_level_num'].median()
    view_level_min = group['view_level_num'].min()

    if view_level_min == 0:
        view_label = 'E'
    elif view_level_min == 3:
        view_label = 'A'  
    elif view_level_min == 1:
        if view_level_median >= 2:
            view_label = 'C'
        else: 
            view_label = 'D'
    elif view_level_min == 2:
        if view_level_median >= 3:
            view_label = 'B'
        elif view_level_median >= 2:
            view_label = 'C'
        else: 
            view_label = 'D'
    #print(f'min={view_level_min} median={view_level_median} label={view_label}')
    return view_label

In [ ]:
# Define the mapping dictionary
mapping_view_level = {
    'insufficient': 0,
    'min': 1,
    'med': 2,
    'high': 3
}

mapping_view_label = {
    'A': 1,
    'B': 0.75,
    'C': 0.5,
    'D': 0.25,
    'E': 0
}

#### Orientation

In [ ]:
def find_orientation_level(room_type, main_orientation):
    if room_type == 'LIVING_ROOM':
        if main_orientation == 'South-West':
            orientation_level = 'high'
        elif main_orientation in ('West', 'South'):
            orientation_level = 'med'
        elif main_orientation in ('North-West', 'South-East'):
            orientation_level = 'low'
        else: 
            orientation_level = 'min'

    elif room_type == 'DINING':
        if main_orientation in ('South-East', 'South'):
            orientation_level = 'high'
        elif main_orientation in ('East', 'South-West'):
            orientation_level = 'med'
        elif main_orientation in ('North-East', 'West'):
            orientation_level = 'low'
        else: 
            orientation_level = 'min'

    elif room_type == 'KITCHEN':
        if main_orientation == 'East':
            orientation_level = 'high'
        elif main_orientation in ('North-East', 'South-East'):
            orientation_level = 'med'
        elif main_orientation in ('North', 'South'):
            orientation_level = 'low'
        else: 
            orientation_level = 'min'

    elif room_type in ('BEDROOM', 'ROOM'):
        if main_orientation == 'South-East':
            orientation_level = 'high'
        elif main_orientation in ('East', 'South'):
            orientation_level = 'med'
        elif main_orientation in ('North-East', 'South-West'):
            orientation_level = 'low'
        else: 
            orientation_level = 'min'

    elif room_type == 'OUTDOOR_SPACE':
        if main_orientation == 'South':
            orientation_level = 'high'
        elif main_orientation in ('South-East','South-West'):
            orientation_level = 'med'
        elif main_orientation in ('North-East', 'East', 'West', 'North-West'):
            orientation_level = 'low'
        else: 
            orientation_level = 'min'

    else: 
        orientation_level = 'nan'

    return orientation_level

In [ ]:
def find_orientation_label(group):
    orientation_level_median = group['orientation_level_num'].median()
    orientation_level_min = group['orientation_level_num'].min()

    if orientation_level_min == 3:
        orientation_label = 'A'  
    elif orientation_level_median >= 3:
        orientation_label = 'B'
    elif orientation_level_median >= 2:
        orientation_label = 'C'
    elif orientation_level_median >= 1:
        orientation_label = 'D'
    else:
        orientation_label = 'E'
    #print(f'min={orientation_level_min} median={orientation_level_median} label={orientation_label}')
    return orientation_label

In [ ]:
# Define the mapping dictionary
mapping_orientation_level = {
    'min': 0,
    'low': 1,
    'med': 2,
    'high': 3
}

mapping_orientation_label = mapping_daylight_label

#### Overall quality

In [ ]:
def find_overall_label_num(day_points, view_points, orientation_points):
    penalty = 1

    if day_points == 0 or view_points == 0 :
        penalty = 0
    
    average = (day_points + view_points + orientation_points) / 3

    overall_label = (average * penalty)

    return overall_label

In [ ]:
mapping_overall_label = {
    'A': (1.0, 0.86),   #10
    'B': (0.85, 0.71),  #15
    'C': (0.70, 0.56),  #15
    'D': (0.55, 0.36),  #20
    'E': (0.35, 0.01),  #35
    'F': (0, 0)
}

def find_overall_label(value):
    for letter, (upper, lower) in mapping_overall_label.items():
        if lower <= value <= upper:
            return letter
    return None  # Return None or another value to handle cases outside the defined ranges

## Combine DF_info

Merge all the seperate csv file togheter into one file

In [ ]:
#Combine all the seperate df_info together into one

# Set the directory where the info CSV files are located
directory = 'C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/Swiss Dataset/SwissDataset_v3.0.0/InfoDataCSV_v2/'

# Get a list of all CSV files in the directory
csv_files = glob.glob(os.path.join(directory, 'SwissDataset_v3.0.0_info_*.csv'))

# Initialize an empty DataFrame to store the combined info data
df_combinedinfo = pd.DataFrame()

# Iterate through each CSV file
for file in csv_files:
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file)
    
    # Append the data to the combined_data DataFrame
    df_combinedinfo = df_combinedinfo.append(data, ignore_index=True)

df_combinedinfo.head()

Remove redudant room types

In [ ]:
print(df_combinedinfo['entity_subtype'].unique())

In [ ]:
# Create a new column 'orientation_distribution' in df_test
df_combinedinfo['orientation_distribution'] = None
df_combinedinfo['orientation_main'] = None

# Create a dictionary to map orientations to indices
orientation_mapping = {'North'          : 0,
                        'North-East'    : 1,
                        'East'          : 2,
                        'South-East'    : 3,
                        'South'         : 4,
                        'South-West'    : 5,
                        'West'          : 6,
                        'North-West'    : 7 }

# Create a mapping for 'OUTSIDE_DOOR' rooms
outside_door_mapping = df_combinedinfo[df_combinedinfo['entity_subtype'] == 'OUTSIDE_DOOR'].set_index('apartment_id')['orientation']

# Iterate over all rows in df_combinedinfo_rooms
for index, row in df_combinedinfo.iterrows():
    room_type = row['entity_subtype']
    
    if room_type in ('LIVING_ROOM', 'ROOM', 'KITCHEN', 'DINING', 'BEDROOM', 'STUDIO'):
        window_count = row['nr_window_sides']
        
        if window_count == 0:
            orientation_distribution = [0] * 8
            main_direction = None
            nr_of_directions = 0

        if window_count > 0:
            orientation_distribution = [0] * 8
            orientation_str = row['orientation_percentage']
            elements = re.findall(r"\['(.*?)' '(.*?)'\]", orientation_str)
            orientation_list = [[element[0], element[1]] for element in elements]

            max_percentage = 0
            main_direction = None

            for orientation, percentage in orientation_list:
                orientation_index = orientation_mapping.get(orientation, -1)
                if orientation_index != -1:
                    orientation_distribution[orientation_index] = round(float(percentage) / 100, 2)
                    count_of_zeros = orientation_distribution.count(0)
                    nr_of_directions = 8 - count_of_zeros

                current_percentage = float(percentage)
                if current_percentage > max_percentage:
                    max_percentage = current_percentage
                    main_direction = orientation

        else:
            orientation_distribution = [0] * 8
            nr_of_directions = 0

        df_combinedinfo.at[index, 'orientation_distribution'] = str(orientation_distribution)
        df_combinedinfo.at[index, 'orientation_main'] = str(main_direction)
        df_combinedinfo.at[index, 'nr_window_sides'] = nr_of_directions
    
    elif room_type == 'OUTDOOR_SPACE':
        app_id = row['apartment_id']
        area_id = row['area_id']
        if app_id in outside_door_mapping:
            main_direction = outside_door_mapping[app_id]
            
            # Check if main_direction is a pandas Series
            if isinstance(main_direction, pd.Series):
                filtered_rows = df_combinedinfo[(df_combinedinfo['apartment_id'] == app_id) & (df_combinedinfo['entity_subtype'] == 'OUTSIDE_DOOR')]
                filtered_rows_with_area_id = filtered_rows[(filtered_rows['door_connection1'] == area_id) | (filtered_rows['door_connection2'] == area_id)]

                number_of_rows = len(filtered_rows_with_area_id)
                
                if number_of_rows == 1:
                    main_direction = filtered_rows_with_area_id['orientation'].iloc[0]
                    df_combinedinfo.at[index, 'orientation_main'] = main_direction

                elif number_of_rows > 1:
                    directions = set(filtered_rows_with_area_id['orientation'].values)
                    number_of_directions = len(directions)

                    if number_of_directions == 1:
                        main_direction = directions.pop()
                        df_combinedinfo.at[index, 'orientation_main'] = main_direction

                    else:
                        if not filtered_rows_with_area_id.empty:
                            # Group the rows by orientation and calculate the total window area for each orientation
                            orientation_window_area = filtered_rows_with_area_id.groupby('orientation')['window_area'].sum()

                            # Find the orientation with the highest total window area
                            main_direction = orientation_window_area.idxmax()
                            df_combinedinfo.at[index, 'orientation_main'] = main_direction
                
            else:
                df_combinedinfo.at[index, 'orientation_main'] = main_direction

In [ ]:
# Store the outdoorspaces 
filtered_data = df_combinedinfo[df_combinedinfo['entity_subtype'] == 'OUTDOOR_SPACE']

# Get the apartment_id values from the filtered data and store them in a list
area_ids = filtered_data['area_id'].tolist()
print(len(area_ids))

# Exclude the rows with the filtered area_ids from the original DataFrame
df_combinedinfo_outdoorspace = df_combinedinfo[df_combinedinfo['area_id'].isin(area_ids)]
df_combinedinfo_outdoorspace = df_combinedinfo_outdoorspace[df_combinedinfo_outdoorspace['entity_subtype'] == 'OUTDOOR_SPACE']

In [ ]:
# Filter rows where 'entity_subtype' is not in 'redundant_entities'
redundant_entities = ['OUTDOOR_SPACE', 'OTHER', 'VOID', 'BATHROOM', 'CORRIDOR', 'STOREROOM', 'CIRCULATION']
filtered_data = df_combinedinfo[df_combinedinfo['entity_subtype'].isin(redundant_entities)]

# Get the apartment_id values from the filtered data and store them in a list
area_ids = filtered_data['area_id'].tolist()
print(len(area_ids))

# Exclude the rows with the filtered area_ids from the original DataFrame
df_combinedinfo1 = df_combinedinfo[~df_combinedinfo['area_id'].isin(area_ids)]

In [ ]:
print(df_combinedinfo1['entity_subtype'].unique())

Make feature orientation distribution usable

Store doors seperately

## Clean DF_geom

In [ ]:
#Create geometry dataframe
#Call CSV file of dataset, and import dataset using Pandas
path_geom = f'C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/Swiss Dataset/SwissDataset_v3.0.0/SwissDataset_v3.0.0_geometries.csv'
Swiss_geom = pd.read_csv(path_geom)

#Create (pandas) dataframe
df_geom = pd.DataFrame(Swiss_geom)

### Clean 0: duplicate rows and features

In [ ]:
#Drop duplicates
df_clean0A_geom = df_geom.drop_duplicates()

#Filter out features
df_clean0B_geom = df_clean0A_geom[df_clean0A_geom["entity_type"] != "feature"]

### Clean 1: multi story appartments

In [ ]:
## Remove the apartments with multiple floors -> outside scope of thesis
# Count the number of unique unit_ids per apartment_id
apartment_unit_counts = df_clean0B_geom.groupby('apartment_id')['unit_id'].nunique()

# Get the apartment_ids with multiple unique unit_ids
apartments_with_multiple_floors = apartment_unit_counts[apartment_unit_counts > 1].index
print(len(set(apartments_with_multiple_floors)))

# Remove rows with apartment_ids that have multiple unique unit_ids
df_clean1_geom = df_clean0B_geom[~df_clean0B_geom['apartment_id'].isin(apartments_with_multiple_floors)]

In [ ]:
# Finding the number of apartments with more than one story in the dataframe
num_app_mult = df_geom['apartment_id'].nunique() - df_clean1_geom['apartment_id'].nunique()
num_room_mult = df_geom['area_id'].nunique() - df_clean1_geom['area_id'].nunique()
print(f'Multiple story apartments: Apartments: {num_app_mult}, Rooms: {num_room_mult}')

In [ ]:
df_combinedinfo1.head()

In [ ]:
df_combinedinfo1.drop_duplicates()

#Store the door info temporarily in a different frame and bring back at the end
df_info_doors = df_combinedinfo1[(df_combinedinfo1['entity_subtype'] == 'DOOR')]
df_combinedinfo2 = df_combinedinfo1[(df_combinedinfo1['entity_subtype'] != 'DOOR')]

# Clean DF_sim

In [ ]:
#Create simulation dataframe
#Call CSV file of dataset, and import dataset using Pandas
path_sim = f'C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/Swiss Dataset/SwissDataset_v3.0.0/SwissDataset_v3.0.0_simulations.csv'
Swiss_sim = pd.read_csv(path_sim)

#Create (pandas) dataframe
df_sim = pd.DataFrame(Swiss_sim)

In [ ]:
df_sim.drop_duplicates()

# Filter rows where 'entity_subtype' is not in 'redundant_entities'
redundant_entities = ['Balcony', 'SunRoom', 'Loggia', 'Bathroom', 'Corridor', 'StorageRoom']
filtered_data = df_sim[df_sim['layout_area_type'].isin(redundant_entities)]

# Get the apartment_id values from the filtered data and store them in a list
area_ids = filtered_data['area_id'].tolist()
print(len(area_ids))

# Exclude the rows with the filtered area_ids from the original DataFrame
df_sim1 = df_sim[~df_sim['area_id'].isin(area_ids)]

# Cleaning final dataframe

In [ ]:
# starting dataframes:
df_combinedinfo2        # dataframe all info from feature creation
df_clean1_geom          # dataframe all geometries
df_sim1                 # dataframe all simulation data

print('continue with these dataframes')

### Clean 2: apartments outside simulation dataframe

In [ ]:
# Get the apartment_id values from the filtered data and store them in sets
apartment_ids_info2_set = set(df_combinedinfo2['apartment_id'])
apartment_ids_sim1_set = set(df_sim1['apartment_id'])

# Find the common apartment_ids using set intersection
common_apartment_ids = apartment_ids_info2_set.intersection(apartment_ids_sim1_set)

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean2_sim = df_sim1[df_sim1['apartment_id'].isin(common_apartment_ids)]
df_clean2_info = df_combinedinfo2[df_combinedinfo2['apartment_id'].isin(common_apartment_ids)]

In [ ]:
# Finding the number of apartments without simulation data in the dataframe
num_app_sim = df_combinedinfo2['apartment_id'].nunique() - df_clean2_info['apartment_id'].nunique()
num_room_sim = df_combinedinfo2['area_id'].nunique() - df_clean2_info['area_id'].nunique()
print(f'Apartments outside simulation dataset: Apartments: {num_app_sim}, Rooms: {num_room_sim}')

### Clean 3: shared windows geometry

In [ ]:
# Get rid of all the apartments with shared windows (1)
# Filter out rows with area_id equal to 1 or 3
filtered_data = df_clean2_info[(df_clean2_info['area_id'] == 1)]

# Get the apartment_id values from the filtered data and store them in a list
apartment_ids = filtered_data['apartment_id'].tolist()

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean3_info = df_clean2_info[~df_clean2_info['apartment_id'].isin(apartment_ids)]

In [ ]:
# Finding the number of apartments with shared windows
num_app_shared = df_clean2_info['apartment_id'].nunique() - df_clean3_info['apartment_id'].nunique()
num_room_shared = df_clean2_info['area_id'].nunique() - df_clean3_info['area_id'].nunique()
print(f'Apartments shared windows: Apartments: {num_app_shared}, Rooms: {num_room_shared}')

In [ ]:
df_clean3_info.head()

### Clean 4: other window geometry problems

In [ ]:
# Get rid of all the apartments with other wrong apartment geomerties (3)
# Filter out rows with area_id equal to 1 or 3
filtered_data = df_clean3_info[(df_clean3_info['area_id'] == 3)]

# Get the apartment_id values from the filtered data and store them in a list
apartment_ids = filtered_data['apartment_id'].tolist()

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean4_info = df_clean3_info[~df_clean3_info['apartment_id'].isin(apartment_ids)]

In [ ]:
#update the geom and simulation dataframes, needed for the next cleaning steps
# Get the apartment_id values from the filtered data and store them in a list
apartment_ids = df_clean4_info['apartment_id'].tolist()

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean4_geom = df_clean1_geom[df_clean1_geom['apartment_id'].isin(apartment_ids)]
df_clean4_sim = df_clean2_sim[df_clean2_sim['apartment_id'].isin(apartment_ids)]

In [ ]:
#create dataframe with only rooms
window_types = ['WINDOW', 'OUTSIDE_DOOR']
df_clean4_info_room = df_clean4_info[~df_clean4_info['entity_subtype'].isin(window_types)]

# Convert the "area_id" column to integers
df_clean4_info_room['area_id'] = df_clean4_info_room['area_id'].astype(float)
df_clean4_info_room['area_id'] = df_clean4_info_room['area_id'].astype('int64')

In [ ]:
# Finding the number of other difficulties in the apartments
num_app_otherprob = df_clean3_info['apartment_id'].nunique() - df_clean4_info['apartment_id'].nunique()
num_room_otherprob = df_clean3_info['area_id'].nunique() - df_clean4_info['area_id'].nunique()
print(f'Apartments other window difficulties: {num_app_otherprob}, Rooms: {num_room_otherprob}')

In [ ]:
df_clean4_info.head()

### Clean 5: Rooms without daylight values but with sky values

In [ ]:
#create needed view columns by combining data
# Select only desired columns for view dataframe
df_views = df_clean4_sim.loc[:, ['area_id', 'apartment_id', 'layout_area_type']]

#Create columns with the four view categories for mean
df_views['view_site_mean'] = df_clean4_sim['view_site_mean']
df_views['view_ground_mean'] = df_clean4_sim[['view_ground_mean', 'view_highways_mean', 'view_railway_tracks_mean','view_tertiary_streets_mean', 'view_secondary_streets_mean', 'view_primary_streets_mean', 'view_pedestrians_mean']].sum(axis=1)
df_views['view_landscape_nature_mean'] = df_clean4_sim[['view_greenery_mean', 'view_water_mean', 'view_mountains_class_2_mean', 'view_mountains_class_3_mean', 'view_mountains_class_4_mean', 'view_mountains_class_5_mean', 'view_mountains_class_6_mean']].sum(axis=1)
df_views['view_landscape_urban_mean'] = df_clean4_sim['view_buildings_mean']
df_views['view_sky_mean'] = df_clean4_sim['view_sky_mean']

#Make percantage
df_views[['view_site_mean', 'view_ground_mean', 'view_landscape_nature_mean', 'view_landscape_urban_mean', 'view_sky_mean']] *= 100

#Find the sum off all the view categories to check that is counts up to 100%
df_views['view_total_mean'] = df_views[['view_site_mean', 'view_ground_mean', 'view_landscape_nature_mean', 'view_landscape_urban_mean', 'view_sky_mean']].sum(axis=1)

#Create columns with the four view categories for median
df_views['view_site_median'] = df_clean4_sim['view_site_median']
df_views['view_ground_median'] = df_clean4_sim[['view_ground_median', 'view_highways_median', 'view_railway_tracks_median','view_tertiary_streets_median', 'view_secondary_streets_median', 'view_primary_streets_median', 'view_pedestrians_median']].sum(axis=1)
df_views['view_landscape_nature_median'] = df_clean4_sim[['view_greenery_median', 'view_water_median', 'view_mountains_class_2_median', 'view_mountains_class_3_median', 'view_mountains_class_4_median', 'view_mountains_class_5_median', 'view_mountains_class_6_median']].sum(axis=1)
df_views['view_landscape_urban_median'] = df_clean4_sim['view_buildings_median']
df_views['view_sky_median'] = df_clean4_sim['view_sky_median']

#Make percantage
df_views[['view_site_median', 'view_ground_median', 'view_landscape_nature_median', 'view_landscape_urban_median', 'view_sky_median']] *= 100


#Create columns with the four view categories for p80
df_views['view_site_p80'] = df_clean4_sim['view_site_p80']
df_views['view_ground_p80'] = df_clean4_sim[['view_ground_p80', 'view_highways_p80', 'view_railway_tracks_p80','view_tertiary_streets_p80', 'view_secondary_streets_p80', 'view_primary_streets_p80', 'view_pedestrians_p80']].sum(axis=1)
df_views['view_landscape_nature_p80'] = df_clean4_sim[['view_greenery_p80', 'view_water_p80', 'view_mountains_class_2_p80', 'view_mountains_class_3_p80', 'view_mountains_class_4_p80', 'view_mountains_class_5_p80', 'view_mountains_class_6_p80']].sum(axis=1)
df_views['view_landscape_urban_p80'] = df_clean4_sim['view_buildings_p80']
df_views['view_sky_p80'] = df_clean4_sim['view_sky_p80']

#Make percantage
df_views[['view_site_p80', 'view_ground_p80', 'view_landscape_nature_p80', 'view_landscape_urban_p80', 'view_sky_p80']] *= 100

#Find the sum off all the view categories to check that is counts up to 100%
df_views['view_total_p80'] = df_views[['view_site_p80', 'view_ground_p80', 'view_landscape_nature_p80', 'view_landscape_urban_p80', 'view_sky_p80']].sum(axis=1)

In [ ]:
#Create needed daylight columns and change to lux
# Multiply all the daylight by 1000 to convert from kilolux to lux
df_sunlight = df_clean4_sim.loc[:, ['area_id', 'apartment_id', 'layout_area_type', 'sun_201803211200_median', 'sun_201806211200_median', 'sun_201812211200_median', 'sun_201803211200_mean', 'sun_201806211200_mean', 'sun_201812211200_mean']]

df_sunlight[['daylight_21Mar1200_median', 'daylight_21Jun1200_median', 'daylight_21Dec1200_median', 'daylight_21Mar1200_mean', 'daylight_21Jun1200_mean', 'daylight_21Dec1200_mean']] = df_sunlight[['sun_201803211200_median', 'sun_201806211200_median', 'sun_201812211200_median', 'sun_201803211200_mean', 'sun_201806211200_mean', 'sun_201812211200_mean']].multiply(1000).round(0)

In [ ]:
# Merge all simulation data to info dataframe
# Merge df info with the view simulation results
df_clean4_info_room = pd.merge(df_clean4_info_room, df_views[['apartment_id', 'area_id', 'view_site_mean', 'view_ground_mean', 'view_landscape_nature_mean', 'view_landscape_urban_mean', 'view_sky_mean','view_total_mean', 'view_site_median', 'view_ground_median', 'view_landscape_nature_median', 'view_landscape_urban_median', 'view_sky_median', 'view_site_p80', 'view_ground_p80', 'view_landscape_nature_p80',	'view_landscape_urban_p80',	'view_sky_p80']], on=['apartment_id', 'area_id'], how='left')

# Merge df info with the daylight simulation results
df_clean4_info_room = pd.merge(df_clean4_info_room, df_sunlight[['apartment_id', 'area_id', 'daylight_21Mar1200_median', 'daylight_21Jun1200_median', 'daylight_21Dec1200_median', 'daylight_21Mar1200_mean', 'daylight_21Jun1200_mean', 'daylight_21Dec1200_mean']], on=['apartment_id', 'area_id'], how='left')

In [ ]:
# Filter out rooms that have no daylight value but a high sky view value 
filtered_data = df_clean4_info_room[(df_clean4_info_room['daylight_21Mar1200_median'] == 0) & (df_clean4_info_room['view_sky_mean'] > 0.477)]

# Get the apartment_id values from the filtered data and store them in a list
apartment_ids = filtered_data['apartment_id'].tolist()

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean5_info_room = df_clean4_info_room[~df_clean4_info_room['apartment_id'].isin(apartment_ids)]

In [ ]:
#Drop rooms without daylight but with view to sky
num_app_daysky = df_clean4_info_room['apartment_id'].nunique() - df_clean5_info_room['apartment_id'].nunique()
num_room_daysky = df_clean4_info_room['area_id'].nunique() - df_clean5_info_room['area_id'].nunique()
print(f'Rooms without daylight with sky view: {num_app_daysky}, Rooms: {num_room_daysky}')

### Clean 6: simulation na values

In [ ]:
# Drop rows with NaN values in the 'sun_201803211200_mean' and 'view_sky' columns
# Store the app_id values of rows to be dropped
dropped_app_ids_labels = df_clean5_info_room[df_clean5_info_room['daylight_21Mar1200_median'].isna() | df_clean5_info_room['daylight_21Jun1200_median'].isna() |df_clean5_info_room['daylight_21Dec1200_median'].isna() | df_clean5_info_room['view_ground_p80'].isna() | df_clean5_info_room['view_sky_p80'].isna()]['apartment_id'].tolist()
dropped_app_ids_features = df_clean5_info_room[df_clean5_info_room['layout_area'].isna() | df_clean5_info_room['room_depth_ratio'].isna() |df_clean5_info_room['nr_window_sides'].isna() | df_clean5_info_room['view_site_p80'].isna() | df_clean5_info_room['view_landscape_nature_p80'].isna() | df_clean5_info_room['view_landscape_urban_p80'].isna() | df_clean5_info_room['elevation'].isna() | df_clean5_info_room['window_floor_ratio'].isna()]['apartment_id'].tolist()
dropped_app_ids = list(set(dropped_app_ids_labels + dropped_app_ids_features))

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean6_info_room = df_clean5_info_room[~df_clean5_info_room['apartment_id'].isin(dropped_app_ids)]

In [ ]:
# Finding the number of rooms without simulation values
num_app_nan = df_clean5_info_room['apartment_id'].nunique() - df_clean6_info_room['apartment_id'].nunique()
num_room_nan = df_clean5_info_room['area_id'].nunique() - df_clean6_info_room['area_id'].nunique()
print(f'Rooms without daylight with sky view: {num_app_nan}, Rooms: {num_room_nan}')

### Clean 7: 

#### Remove outliers for sky view p80 and daylight median and filter out all appartments

In [ ]:
# Assuming df_room_complete4 is your DataFrame
data_sky = df_clean6_info_room['view_sky_p80']

# Calculate the mean and standard deviation for the 'sky' column
mean_sky = data_sky.mean()
std_sky = data_sky.std()

# Create a Boolean mask for outliers in the 'sky_21Mar1200_median' column a threshold for outliers (e.g., 3 times the standard deviation)
outlier_mask_sky = (data_sky - mean_sky).abs() > 3 * std_sky

# Filter the DataFrame to keep only rows without outliers in the 'sky_21Mar1200_median' column
sky_outlier_rooms = df_clean6_info_room[outlier_mask_sky]

# Get the apartment_id values from the filtered data and store them in a list
apartment_ids = sky_outlier_rooms['apartment_id'].tolist()
print(len(set(apartment_ids)))

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean7_info_room_sky = df_clean6_info_room[~df_clean6_info_room['apartment_id'].isin(apartment_ids)]

In [ ]:
# Assuming df_room_complete4 is your DataFrame
data_ground = df_clean6_info_room['view_ground_p80']

# Calculate the mean and standard deviation for the 'ground' column
mean_ground = data_ground.mean()
std_ground = data_ground.std()

# Create a Boolean mask for outliers in the 'ground_21Mar1200_median' column a threshold for outliers (e.g., 3 times the standard deviation)
outlier_mask_ground = (data_ground - mean_ground).abs() > 3 * std_ground

# Filter the DataFrame to keep only rows without outliers in the 'ground_21Mar1200_median' column
ground_outlier_rooms = df_clean6_info_room[outlier_mask_ground]

# Get the apartment_id values from the filtered data and store them in a list
apartment_ids = ground_outlier_rooms['apartment_id'].tolist()
print(len(set(apartment_ids)))

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean7_info_room_ground = df_clean6_info_room[~df_clean6_info_room['apartment_id'].isin(apartment_ids)]

In [ ]:
# Assuming df_room_complete4 is your DataFrame
data_daylight_Mar = df_clean6_info_room['daylight_21Mar1200_median']

# Calculate the mean and standard deviation for the 'daylight_21Mar1200_median' column
mean_daylight_Mar = data_daylight_Mar.mean()
std_daylight_Mar = data_daylight_Mar.std()

# Create a Boolean mask for outliers in the 'daylight_21Mar1200_median' column a threshold for outliers (e.g., 3 times the standard deviation)
outlier_mask_daylight_Mar = (data_daylight_Mar - mean_daylight_Mar).abs() > 3 * std_daylight_Mar

# Filter the DataFrame to keep only rows without outliers in the 'daylight_21Mar1200_median' column
daylight_Mar_outlier_rooms = df_clean6_info_room[outlier_mask_daylight_Mar]

# Get the apartment_id values from the filtered data and store them in a list
apartment_ids = daylight_Mar_outlier_rooms['apartment_id'].tolist()
print(len(set(apartment_ids)))

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean7_info_room_daylight_Mar = df_clean6_info_room[~df_clean6_info_room['apartment_id'].isin(apartment_ids)]

In [ ]:
# Assuming df_room_complete4 is your DataFrame
data_daylight_Jun = df_clean6_info_room['daylight_21Jun1200_median']

# Calculate the mean and standard deviation for the 'daylight_21Jun1200_median' column
mean_daylight_Jun = data_daylight_Jun.mean()
std_daylight_Jun = data_daylight_Jun.std()

# Create a Boolean mask for outliers in the 'daylight_21Jun1200_median' column a threshold for outliers (e.g., 3 times the standard deviation)
outlier_mask_daylight_Jun = (data_daylight_Jun - mean_daylight_Jun).abs() > 3 * std_daylight_Jun

# Filter the DataFrame to keep only rows without outliers in the 'daylight_21Jun1200_median' column
daylight_Jun_outlier_rooms = df_clean6_info_room[outlier_mask_daylight_Jun]

# Get the apartment_id values from the filtered data and store them in a list
apartment_ids = daylight_Jun_outlier_rooms['apartment_id'].tolist()
print(len(set(apartment_ids)))

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean7_info_room_daylight_Jun = df_clean6_info_room[~df_clean6_info_room['apartment_id'].isin(apartment_ids)]

In [ ]:
# Assuming df_room_complete4 is your DataFrame
data_daylight_Dec = df_clean6_info_room['daylight_21Dec1200_median']

# Calculate the mean and standard deviation for the 'daylight_21Dec1200_median' column
mean_daylight_Dec = data_daylight_Dec.mean()
std_daylight_Dec = data_daylight_Dec.std()

# Create a Boolean mask for outliers in the 'daylight_21Dec1200_median' column a threshold for outliers (e.g., 3 times the standard deviation)
outlier_mask_daylight_Dec = (data_daylight_Dec - mean_daylight_Dec).abs() > 3 * std_daylight_Dec

# Filter the DataFrame to keep only rows without outliers in the 'daylight_21Dec1200_median' column
daylight_Dec_outlier_rooms = df_clean6_info_room[outlier_mask_daylight_Dec]

# Get the apartment_id values from the filtered data and store them in a list
apartment_ids = daylight_Dec_outlier_rooms['apartment_id'].tolist()
print(len(set(apartment_ids)))

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean7_info_room_daylight_Dec = df_clean6_info_room[~df_clean6_info_room['apartment_id'].isin(apartment_ids)]

In [ ]:
# Get the apartment_id values from the filtered data and store them in sets
apartment_ids_clean7_sky = set(df_clean7_info_room_sky['apartment_id'])
apartment_ids_clean7_ground = set(df_clean7_info_room_ground['apartment_id'])
apartment_ids_clean7_daylight_Mar = set(df_clean7_info_room_daylight_Mar['apartment_id'])
apartment_ids_clean7_daylight_Jun = set(df_clean7_info_room_daylight_Jun['apartment_id'])
apartment_ids_clean7_daylight_Dec = set(df_clean7_info_room_daylight_Dec['apartment_id'])

# Find the common apartment_ids using set intersection
common_apartment_ids = (
    apartment_ids_clean7_sky &
    apartment_ids_clean7_ground &
    apartment_ids_clean7_daylight_Mar &
    apartment_ids_clean7_daylight_Jun &
    apartment_ids_clean7_daylight_Dec
)

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean7_info_room_temp = df_clean6_info_room[df_clean6_info_room['apartment_id'].isin(common_apartment_ids)]

In [ ]:
# Merge the two dataframes based on 'apartment_id' and 'area_id'
df_merged_info_room_temp = df_clean7_info_room_temp.merge(df_clean4_geom[['apartment_id', 'area_id', 'building_id', 'floor_id', 'height']], on=['apartment_id', 'area_id'], how='left')

# Update the 'building_id' column in df_clean_info_room7 with values from df_clean7_geom
df_clean7_info_room_temp['height'] = df_merged_info_room_temp['height']

In [ ]:
#Remove additional outliers of ground view, room area & room height
# Filter rows where [view_ground_p80] is greater than 15
filtered_df = df_clean7_info_room_temp[(df_clean7_info_room_temp['view_landscape_urban_p80'] > 12) | (df_clean7_info_room_temp['view_landscape_nature_p80'] > 12) | (df_clean7_info_room_temp['layout_area'] > 100) | (df_clean7_info_room_temp['height'] > 3.4)]

# Get the [apartment_id] values from the filtered DataFrame
apartment_ids = filtered_df['apartment_id'].tolist()

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean7_info_room = df_clean7_info_room_temp[~df_clean7_info_room_temp['apartment_id'].isin(apartment_ids)]

In [ ]:
# Finding the number of apartments that are daylight and/or sky view outliers
num_app_outliers = df_clean6_info_room['apartment_id'].nunique() - df_clean7_info_room['apartment_id'].nunique()
num_room_outliers = df_clean6_info_room['area_id'].nunique() - df_clean7_info_room['area_id'].nunique()
print(f'Daylight and sky view outliers: {num_app_outliers}, Rooms: {num_room_outliers}')

In [ ]:
df_clean6_info_room['data_ground_current'] = MinMaxScaler().fit_transform(df_clean6_info_room['view_ground_p80'].values.reshape(-1, 1))
df_clean6_info_room['data_sky_current'] = MinMaxScaler().fit_transform(df_clean6_info_room['view_sky_p80'].values.reshape(-1, 1))

df_clean6_info_room['data_dayMar_current'] = MinMaxScaler().fit_transform(df_clean6_info_room['daylight_21Mar1200_median'].values.reshape(-1, 1))
df_clean6_info_room['data_dayJun_current'] = MinMaxScaler().fit_transform(df_clean6_info_room['daylight_21Jun1200_median'].values.reshape(-1, 1))
df_clean6_info_room['data_dayDec_current'] = MinMaxScaler().fit_transform(df_clean6_info_room['daylight_21Dec1200_median'].values.reshape(-1, 1))

In [ ]:
df_clean7_info_room['data_ground_NOoutliers'] = MinMaxScaler().fit_transform(df_clean7_info_room['view_ground_p80'].values.reshape(-1, 1))
df_clean7_info_room['data_sky_NOoutliers'] = MinMaxScaler().fit_transform(df_clean7_info_room['view_sky_p80'].values.reshape(-1, 1))

df_clean7_info_room['data_dayMar_NOoutliers'] = MinMaxScaler().fit_transform(df_clean7_info_room['daylight_21Mar1200_median'].values.reshape(-1, 1))
df_clean7_info_room['data_dayJun_NOoutliers'] = MinMaxScaler().fit_transform(df_clean7_info_room['daylight_21Jun1200_median'].values.reshape(-1, 1))
df_clean7_info_room['data_dayDec_NOoutliers'] = MinMaxScaler().fit_transform(df_clean7_info_room['daylight_21Dec1200_median'].values.reshape(-1, 1))

In [ ]:
boxplot_path = f'C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/10_Images_04Dataset_data/Boxplots_v4/'
scatterplot_path = f'C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/10_Images_04Dataset_data/Scatterplots_v4/'

In [ ]:
# Define the columns you want to plot - current state
data_ground_current = df_clean6_info_room['data_ground_current']
data_sky_current = df_clean6_info_room['data_sky_current']

data_dayMar_current = df_clean6_info_room['data_dayMar_current']
data_dayJun_current = df_clean6_info_room['data_dayJun_current']
data_dayDec_current = df_clean6_info_room['data_dayDec_current']

# state without outliers
data_ground_NOoutliers = df_clean7_info_room['data_ground_NOoutliers']
data_sky_NOoutliers = df_clean7_info_room['data_sky_NOoutliers']

data_dayMar_NOoutliers = df_clean7_info_room['data_dayMar_NOoutliers']
data_dayJun_NOoutliers = df_clean7_info_room['data_dayJun_NOoutliers']
data_dayDec_NOoutliers = df_clean7_info_room['data_dayDec_NOoutliers']

In [ ]:
# Define the columns you want to plot
data = [data_ground_NOoutliers, data_sky_NOoutliers, data_dayDec_NOoutliers, data_dayJun_NOoutliers, data_dayMar_NOoutliers, data_ground_current, data_sky_current, data_dayDec_current, data_dayJun_current, data_dayMar_current]

# Create groups by view type
box_labels = ['without outliers', 'with outliers']
names_colors = ['daylight 21 Mar median', 'daylight 21 Jun median', 'daylight 21 Dec median', 'sky view p80', 'ground view p80']

# Define colors for each view
colors = ['#084C61', '#65949F', '#903C59', '#D7A6B3', '#B0647E', '#084C61', '#65949F', '#903C59', '#D7A6B3', '#B0647E']
colors1 = ['#B0647E', '#D7A6B3', '#903C59', '#65949F', '#084C61']

# Set x-positions for boxes
position = [0.2, 0.4, 0.6, 0.8, 1, 1.4, 1.6, 1.8, 2.0, 2.2]  # Adjust the positions as needed
position1 = [0.6, 1.8]  # Adjust the positions as needed

# Create a figure and axis objects
fig, ax = plt.subplots(figsize=(18, 6))

# Plot
bp = plt.boxplot(data, widths=0.15, patch_artist=True, vert=False, positions=position)

# Set the colors for the faces of the boxes
for i, box in enumerate(bp['boxes']):
    box.set(facecolor=colors[i])

# Make the median lines more visible
plt.setp(bp['medians'], color='black')

# Customize outliers, set smaller and transparent outliers
for flier in bp['fliers']:
    flier.set(marker='o', markerfacecolor='#808081', markersize=3, alpha=0.2)  

# Set the labels and title
ax.set_title('Normalised label values')

# Set tick locations and labels
ax.set_yticks(position1)
ax.set_yticklabels(box_labels, rotation=90, ha='center', va='center')

# Set x-axis limit
ax.set_xlim([-0.01, 1.01])  # Adjust the x-axis limit as needed
ax.set_ylim([0, 2.4])  # Adjust the x-axis limit as needed

# Create custom legend at the top right corner
legend_handles = []
for i, name in enumerate(names_colors):
    legend_handles.append(mpatches.Patch(color=colors1[i], label=name))

# Adjust the legend position using bbox_to_anchor
plt.legend(handles=legend_handles, title='ML label', loc='upper right')

# Safe & show the plot
plt.savefig(f'{boxplot_path}boxplot_removing_outliers.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Data
x_data1 = df_clean6_info_room['daylight_21Mar1200_median']
x_data2 = df_clean7_info_room['daylight_21Mar1200_median']
y_data1 = df_clean6_info_room['view_sky_p80']
y_data2 = df_clean7_info_room['view_sky_p80']
print('plot1: daylight max:', x_data1.max(), ' sky view max:', y_data1.max())
print('plot2: daylight max:', x_data2.max(), ' sky view max:', y_data2.max())

# Titles and labels
title1 = 'Sky view vs. daylight with outliers'
title2 = 'Sky view vs. daylight without outliers'
x_name1 = 'Daylight at 21st March at 12:00 median [lx]'
x_name2 = 'Daylight at 21st March at 12:00 median [lx]'
y_name1 = 'View to sky p80 [%]'
y_name2 = 'View to sky p80 [%]'

# Create a figure with two subplots in a single row
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Scatter plots
axes[0].scatter(x_data1, y_data1, s=20, color='#084C61', alpha=0.1)
axes[1].scatter(x_data2, y_data2, s=20, color='#084C61', alpha=0.1)

# Fit and plot linear regression lines
for ax, x_data, y_data, x_name, y_name, title in zip(axes, [x_data1, x_data2], [y_data1, y_data2], [x_name1, x_name2], [y_name1, y_name2], [title1, title2]):
    z = np.polyfit(x_data, y_data, 1)
    p = np.poly1d(z)
    
    # Customize x and y-axis ranges for each subplot
    if title == 'Sky view vs. daylight with outliers':  # Replace 'x_name1' with the appropriate condition
        x_range = (0, 9000)  # Adjust the x-axis range as needed
        y_range = (0, 16)    # Adjust the y-axis range as needed
    else:
        x_range = (0, 1700)
        y_range = (0, 5.5)
    
    ax.plot(x_range, p(x_range), color='#F7D08A', label='Linear regression')
    
    ax.set_title(title)
    ax.set_xlabel(x_name)
    ax.set_ylabel(y_name)
    
    ax.set_xlim(x_range)  # Set the x-axis range
    ax.set_ylim(y_range)  # Set the y-axis range

# Adjust spacing between subplots
plt.tight_layout()

# Save and show the plot
plt.savefig(f'{scatterplot_path}scatterplot__removing_outliers.png', dpi=300)
plt.show()


In [ ]:
# Get a list of the final apartment_ids
apartment_ids = df_clean7_info_room['apartment_id'].tolist()

# Only include the final apartments for the dataframes geometry, simulations, and fullt df_info
df_clean7_geom = df_clean4_geom[df_clean4_geom['apartment_id'].isin(apartment_ids)]
df_clean7_sim = df_clean4_sim[df_clean4_sim['apartment_id'].isin(apartment_ids)]
df_clean7_sim = df_clean7_sim.dropna(subset=['sun_201803211200_median', 'view_sky_p80'], axis=0)

# Assuming df_clean4_info and df_info_doors are your DataFrames
df_clean4_info = pd.concat([df_clean4_info, df_info_doors], ignore_index=True)
df_clean7_info = df_clean4_info[df_clean4_info['apartment_id'].isin(apartment_ids)]

In [ ]:
# Merge the two dataframes based on 'apartment_id' and 'area_id'
df_merged_info = df_clean7_info.merge(df_clean7_geom[['apartment_id', 'area_id', 'building_id', 'floor_id', 'height']], on=['apartment_id', 'area_id'], how='left')

# Update the 'building_id' column in df_clean_info7 with values from df_clean7_geom
df_clean7_info['building_id'] = df_merged_info['building_id']
df_clean7_info['floor_id'] = df_merged_info['floor_id']
df_clean7_info['height'] = df_merged_info['height']

In [ ]:
# Reorder the columns to place the new columns next to 'site_id'
column_order = ['site_id', 'building_id', 'floor_id', 'apartment_id', 'area_id', 'entity_subtype', 'geometry', 'elevation', 'height', 
                'door_connection1', 'door_connection2', 'orientation', 'window_height', 'window_length', 'window_area']
df_clean8_info = df_clean7_info[column_order]

In [ ]:
# Merge the two dataframes based on 'apartment_id' and 'area_id'
df_merged_info_room = df_clean7_info_room.merge(df_clean7_geom[['apartment_id', 'area_id', 'building_id', 'floor_id']], on=['apartment_id', 'area_id'], how='left')

# Update the 'building_id' column in df_clean_info_room7 with values from df_clean7_geom
df_clean7_info_room['building_id'] = df_merged_info_room['building_id']
df_clean7_info_room['floor_id'] = df_merged_info_room['floor_id']

In [ ]:
df_clean7_info_room['view_landscape_p80'] = df_clean7_info_room['view_landscape_nature_p80'] + df_clean7_info_room['view_landscape_urban_p80']

In [ ]:
# Reorder the columns to place the new columns next to 'site_id'
column_order = ['site_id', 'building_id', 'floor_id', 'apartment_id', 'area_id', 'entity_subtype', 'geometry', 'elevation', 'height', 
                'orientation', 'orientation_percentage', 'orientation_distribution', 'orientation_main',
                'window_area', 'wall_area', 'window_wall_ratio', 'room_depth_ratio', 'layout_area', 'window_floor_ratio',
                'view_site_mean', 'view_ground_mean', 'view_landscape_nature_mean', 'view_landscape_urban_mean', 'view_sky_mean', 'view_total_mean', 
                'view_site_median', 'view_ground_median', 'view_landscape_nature_median', 'view_landscape_urban_median', 'view_sky_median', 
                'view_site_p80', 'view_ground_p80', 'view_landscape_p80', 'view_landscape_nature_p80', 'view_landscape_urban_p80', 'view_sky_p80', 
                'daylight_21Mar1200_mean', 'daylight_21Jun1200_mean', 'daylight_21Dec1200_mean',
                'daylight_21Mar1200_median', 'daylight_21Jun1200_median', 'daylight_21Dec1200_median',
                'data_ground_NOoutliers', 'data_sky_NOoutliers', 'data_dayMar_NOoutliers', 'data_dayJun_NOoutliers', 'data_dayDec_NOoutliers']
df_clean8_info_room = df_clean7_info_room[column_order]

In [ ]:
# Convert the 'window_height' column to float data type, handling NaN values
df_clean8_info['orientation'] = pd.to_numeric(df_clean8_info['orientation'], errors='coerce').astype(str)

In [ ]:
# Assuming df_clean4_info is your DataFrame
df_clean8_info_room = df_clean7_info_room.rename(columns={
    'data_ground_NOoutliers': 'view_ground_p80_normalised',
    'data_sky_NOoutliers': 'view_sky_p80_normalised',
    'data_dayMar_NOoutliers': 'daylight_21Mar1200_median_normalised', 
    'data_dayJun_NOoutliers': 'daylight_21Jun1200_median_normalised',
    'data_dayDec_NOoutliers': 'daylight_21Dec1200_median_normalised'
})

In [ ]:
df_clean8_info_room.head()

In [ ]:
print(df_clean8_info_room['entity_subtype'].unique())

# Find performance levels

In [ ]:
# List of columns you want to keep in the new DataFrame
columns_to_keep = ['apartment_id', 'site_id', 'area_id', 'entity_subtype', 'orientation_main','daylight_21Mar1200_median', 'daylight_21Jun1200_median', 'daylight_21Dec1200_median', 'view_ground_p80', 'view_landscape_p80', 'view_sky_p80']

# Create the new DataFrame by selecting the specified columns
df_clean8_performance = df_clean8_info_room[columns_to_keep].copy()

### Daylight performance

In [ ]:
# daylight levels for the rooms
# Calculate the median along the specified columns and store it in a new column
df_clean8_performance['daylight_median'] = df_clean8_performance[['daylight_21Mar1200_median', 'daylight_21Jun1200_median', 'daylight_21Dec1200_median']].median(axis=1)

# Apply the function to the DataFrame and create a new column 'daylight_level'
df_clean8_performance['daylight_level'] = df_clean8_performance.apply(
    lambda row: find_daylight_level(row['daylight_median'], row['entity_subtype']),
    axis=1
)

# Apply the function to the DataFrame and create a new column 'daylight_level'
df_clean8_performance['daylight_level_num'] = df_clean8_performance['daylight_level'].map(mapping_daylight_level)

In [ ]:
# find the daylight labels for each apartment
# Step 1: Group by 'apartment_id'
grouped = df_clean8_performance.groupby('apartment_id')

# Step 2: Calculate the median and min values for each apartment group
daylight_level_median = grouped['daylight_level_num'].median()
daylight_level_min = grouped['daylight_level_num'].min()

# Create a dictionary to store 'daylight_label' for each apartment
apartment_daylight_labels = {}

# Step 3: Apply the function to each apartment group and update the dictionary
for group_name, group_data in grouped:
    daylight_label = find_daylight_label(group_data)
    apartment_daylight_labels[group_name] = daylight_label

# Step 4: Update 'daylight_label' in the original DataFrame for the rooms that belong to the apartment
df_clean8_performance['daylight_label'] = df_clean8_performance['apartment_id'].map(apartment_daylight_labels)

In [ ]:
# Apply the function to the DataFrame and create a new column 'daylight_level'
df_clean8_performance['daylight_label_num'] = df_clean8_performance['daylight_label'].map(mapping_daylight_label)

### View performance

In [ ]:
df_clean8_performance['landscape_visible'] = df_clean8_performance['view_landscape_p80'].apply(lambda x: 'yes' if x >= 0.477 else 'no')
df_clean8_performance['visible_view_layers'] = df_clean8_performance.apply(count_visible_views, axis=1)

# Apply the function to the DataFrame and create a new column 'view_level'
df_clean8_performance['view_level'] = df_clean8_performance.apply(
    lambda row: find_view_level(row['landscape_visible'], row['visible_view_layers']),
    axis=1
)

# Apply the function to the DataFrame and create a new column 'view_level'
df_clean8_performance['view_level_num'] = df_clean8_performance['view_level'].map(mapping_view_level)

In [ ]:
# find the view labels for each apartment
# Step 1: Group by 'apartment_id'
grouped = df_clean8_performance.groupby('apartment_id')

# Step 2: Calculate the median and min values for each apartment group
view_level_median = grouped['view_level_num'].median()
view_level_min = grouped['view_level_num'].min()

# Create a dictionary to store 'view_label' for each apartment
apartment_view_labels = {}

# Step 3: Apply the function to each apartment group and update the dictionary
for group_name, group_data in grouped:
    view_label = find_view_label(group_data)
    apartment_view_labels[group_name] = view_label

# Step 4: Update 'view_label' in the original DataFrame for the rooms that belong to the apartment
df_clean8_performance['view_label'] = df_clean8_performance['apartment_id'].map(apartment_view_labels)

In [ ]:
# Apply the function to the DataFrame and create a new column 'view_level'
df_clean8_performance['view_label_num'] = df_clean8_performance['view_label'].map(mapping_view_label)

### Orientation performance

In [ ]:
# Get the apartment_id values from the filtered data and store them in a list
apartment_ids = df_clean8_performance['apartment_id'].tolist()

# Exclude the rows with the filtered apartment_ids from the original DataFrame
df_clean8_outdoorspace = df_combinedinfo_outdoorspace[df_combinedinfo_outdoorspace['apartment_id'].isin(apartment_ids)]

In [ ]:
# Get the common columns that exist in both DataFrames
common_columns = list(set(df_clean8_performance.columns) & set(df_clean8_outdoorspace.columns))

# Select the common columns from df_clean8_outdoorspace
selected_columns = df_clean8_outdoorspace[common_columns]

# Concatenate the selected columns from df_clean8_outdoorspace under df_clean8_performance
df_clean8_performance = pd.concat([df_clean8_performance, selected_columns], ignore_index=True)

In [ ]:
# Apply the function to the DataFrame and create a new column 'orientation_level'
df_clean8_performance['orientation_level'] = df_clean8_performance.apply(
    lambda row: find_orientation_level(row['entity_subtype'], row['orientation_main']),
    axis=1
)

# Apply the function to the DataFrame and create a new column 'orientation_level'
df_clean8_performance['orientation_level_num'] = df_clean8_performance['orientation_level'].map(mapping_orientation_level)

In [ ]:
# find the orientation labels for each apartment
# Step 1: Group by 'apartment_id'
grouped = df_clean8_performance.groupby('apartment_id')

# Step 2: Calculate the median and min values for each apartment group
orientation_level_median = grouped['orientation_level_num'].median()
orientation_level_min = grouped['orientation_level_num'].min()

# Create a dictionary to store 'orientation_label' for each apartment
apartment_orientation_labels = {}

# Step 3: Apply the function to each apartment group and update the dictionary
for group_name, group_data in grouped:
    orientation_label = find_orientation_label(group_data)
    apartment_orientation_labels[group_name] = orientation_label

# Step 4: Update 'orientation_label' in the original DataFrame for the rooms that belong to the apartment
df_clean8_performance['orientation_label'] = df_clean8_performance['apartment_id'].map(apartment_orientation_labels)

In [ ]:
# Apply the function to the DataFrame and create a new column 'orientation_level'
df_clean8_performance['orientation_label_num'] = df_clean8_performance['orientation_label'].map(mapping_orientation_label)

### Overall apartment performance

In [ ]:
df_clean8_performance['overall_label_num'] = df_clean8_performance.apply(
    lambda row: find_overall_label_num(row['daylight_label_num'], row['view_label_num'], row['orientation_label_num']),
    axis=1
)

In [ ]:
df_clean8_performance['overall_label'] = df_clean8_performance.apply(
    lambda row: find_overall_label(row['overall_label_num']),
    axis=1
)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already computed the value counts
filtered_df = df_clean8_performance.groupby('apartment_id').first().reset_index()
value_counts = filtered_df['overall_label'].value_counts()

# Create a bar plot
plt.figure(figsize=(8, 6))
value_counts.plot(kind='bar', color='skyblue')
print(value_counts)
# Customize the plot
plt.title('Value Counts of overall Labels')
plt.xlabel('overall Labels')
plt.ylabel('Count')

# Show the plot
plt.xticks(rotation=45)  # Rotate x-axis labels for readability
plt.show()


In [ ]:
df_clean8_performance.head()

In [ ]:
# Reset the index if necessary
df_clean8_info_room.reset_index(drop=True, inplace=True)
df_clean8_info.reset_index(drop=True, inplace=True)
df_clean8_performance.reset_index(drop=True, inplace=True)

In [ ]:
df_clean8_info_room.nunique()

# Stats of lost apartments

In [ ]:
# Finding the number of duplicate rows in the dataframe
num_rows_dub = df_geom.shape[0] - df_clean0A_geom.shape[0]
print("Dubplicate rows:", num_rows_dub, 'rows dropped')

# Finding the number of feature rows in the dataframe
num_rows_feat = df_clean0A_geom.shape[0] - df_clean0B_geom.shape[0]
print("Feature rows:", num_rows_feat, 'rows dropped')

In [ ]:
# Create a list of dictionaries to hold the information
data = []

# Finding the number of original apartments
num_app_OG = df_clean0B_geom['apartment_id'].nunique()
num_room_OG = df_clean0B_geom['area_id'].nunique()
data.append({'Description': 'Total number of apartments', 'Apartments': num_app_OG, 'Rooms': num_room_OG})

# Finding the number of starting apartments
num_app_start = df_combinedinfo1['apartment_id'].nunique()
num_room_start = df_combinedinfo1['area_id'].nunique()
data.append({'Description': 'Total number without redundant room types', 'Apartments': num_app_start, 'Rooms': num_room_start})

# Finding the number of apartments with more than one story in the dataframe
data.append({'Description': 'Multiple story apartments', 'Apartments': num_app_mult, 'Rooms': num_room_mult})

# Finding the number of apartments without simulation data in the dataframe
data.append({'Description': 'Apartments without simulation results', 'Apartments': num_app_sim, 'Rooms': num_room_sim})

# Finding the number of apartments with shared windows
data.append({'Description': 'Apartments shared windows', 'Apartments': num_app_shared, 'Rooms': num_room_shared})

# Finding the number of other difficulties in the apartments
data.append({'Description': 'Apartments other difficulties', 'Apartments': num_app_otherprob, 'Rooms': num_room_otherprob})

#Drop rooms without daylight but with view to sky
data.append({'Description': 'Rooms without daylight with sky view', 'Apartments': num_app_daysky, 'Rooms': num_room_daysky})

# Finding the number of rooms without simulation values
data.append({'Description': 'Rooms with nan simulation values', 'Apartments': num_app_nan, 'Rooms': num_room_nan})

# Finding the number of apartments that are daylight and/or sky view outliers
data.append({'Description': 'Daylight and sky view outliers', 'Apartments': num_app_outliers, 'Rooms': num_room_outliers})

# Finding the number of final apartments
num_app_final = df_clean8_info_room['apartment_id'].nunique()
num_room_final = df_clean8_info_room['area_id'].nunique()
data.append({'Description': 'Total number of final apartments', 'Apartments': num_app_final, 'Rooms': num_room_final})

# Create a DataFrame from the list of dictionaries
df_results = pd.DataFrame(data)

df_results

# Safe dataframes

In [ ]:
#'''
# Write the combined data to a new CSV file
df_clean8_info.to_csv('C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/Swiss Dataset/SwissDataset_v3.0.0/SwissDataset_v3.0.0_clean_v3_info.csv', index=False)
df_clean7_geom.to_csv('C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/Swiss Dataset/SwissDataset_v3.0.0/SwissDataset_v3.0.0_clean_v3_geom.csv', index=False)
df_clean7_sim.to_csv('C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/Swiss Dataset/SwissDataset_v3.0.0/SwissDataset_v3.0.0_clean_v3_sim.csv', index=False)
'''

''' 
df_clean8_info_room.to_csv('C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/Swiss Dataset/SwissDataset_v3.0.0/SwissDataset_v3.0.0_clean_v3_room.csv', index=False)
df_clean8_performance.to_csv('C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/Swiss Dataset/SwissDataset_v3.0.0/SwissDataset_v3.0.0_clean_v3_performance.csv', index=False)
#'''